# Segmenting and Clustering Neighborhoods in Toronto
## part1

In [1]:
#importing libraries for data scraping
from bs4 import BeautifulSoup
import requests # library to handle requests

In [2]:
#define webpage, and get its contents
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
print(response.text[0:500])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","w


In [3]:
#get the text part of the dataframe..
html_doc=response.text
soup = BeautifulSoup(html_doc, 'html.parser')

In [4]:
#for testing uncomment - print tesult - takes to much space
#print(soup.prettify())

In [5]:
#there are 3 or more tables on the page, but the one of interest is of class 'wikitable sortable' so I take it
#for link in soup.find_all('table'):
#    print(link.get('tbody'))
##<table class="wikitable sortable">
##       <tbody>
table=soup.find('table',attrs={'class':'wikitable sortable'})

In [6]:
#get column names
column_data=table.find_all('th')#this is where the column names are defined
column_data

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [7]:
#get rid of tags
column_names=[]
for columns in column_data:
    column_names.append(columns.text.strip())
print(column_names)

['Postcode', 'Borough', 'Neighbourhood']


In [8]:
#getting the table data
table_data=[]
rows=table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    table_data.append([ele for ele in cols if ele]) # Get rid of empty values
#column_data=table.find_all('td')
#for columns in column_data:
#    table_data.append(columns.text)
print(table_data)

[[], ['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park", 'Not assigned'], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Etobicoke', 'Islington'], ['M9B', 'Etobicoke', 'Martin

In [9]:
#I am going to need this
import pandas as pd
import numpy as np

In [10]:
#create pandas dataframe
canada_postal_codes = pd.DataFrame(columns=column_names, data=table_data)
canada_postal_codes.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [11]:
#filterout nulls
canada_postal_codes=canada_postal_codes[canada_postal_codes['Borough'].notnull()]
#filter out 'Not assigned'
canada_postal_codes=canada_postal_codes[canada_postal_codes['Borough']!='Not assigned']
canada_postal_codes.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [12]:
#If a cell has a borough but a Not assigned neighborhood...
canada_postal_codes.loc[(canada_postal_codes['Neighbourhood']=='Not assigned'),'Neighbourhood']=canada_postal_codes.loc[(canada_postal_codes['Neighbourhood']=='Not assigned'),'Borough']
canada_postal_codes.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [13]:
#group by; first i get the unique values for Postcode and Borough
canada_postal_codes2=canada_postal_codes.drop(columns=['Neighbourhood'])
canada_postal_codes2=canada_postal_codes2.drop_duplicates()
canada_postal_codes2['Neighbourhood']= np.nan
canada_postal_codes2.shape
canada_postal_codes2.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,NaN
4,M4A,North York,NaN
5,M5A,Downtown Toronto,NaN
7,M6A,North York,NaN
9,M7A,Queen's Park,NaN


In [14]:
#I am native to PLSQL, so this is my way of making an equivalent to stragg function
for index, row in canada_postal_codes.iterrows():
    canada_postal_codes2.loc[(canada_postal_codes2['Borough']==row['Borough']) & (canada_postal_codes2['Postcode']==row['Postcode']),'Neighbourhood']=canada_postal_codes2.loc[(canada_postal_codes2['Borough']==row['Borough']) & (canada_postal_codes2['Postcode']==row['Postcode']),'Neighbourhood'].fillna('')+row['Neighbourhood']+', '
canada_postal_codes2.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,"Parkwoods,"
4,M4A,North York,"Victoria Village,"
5,M5A,Downtown Toronto,"Harbourfront, Regent Park,"
7,M6A,North York,"Lawrence Heights, Lawrence Manor,"
9,M7A,Queen's Park,"Queen's Park,"


In [15]:
#clean the data get rid of ", " at the end
canada_postal_codes2['Neighbourhood']=canada_postal_codes2['Neighbourhood'].str.slice(stop=-2)
canada_postal_codes2.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Harbourfront, Regent Park"
7,M6A,North York,"Lawrence Heights, Lawrence Manor"
9,M7A,Queen's Park,Queen's Park


In [16]:
#reseting the index to get the same result
canada_postal_codes2.reset_index(drop=True, inplace=True)
canada_postal_codes2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [17]:
#shape
canada_postal_codes2.shape

(103, 3)

# part2

In [18]:
#get the CSV file
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [19]:
#read it into a dataframe
Geospatial_Coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
Geospatial_Coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
toronto_neighborhoods = pd.concat([canada_postal_codes2, Geospatial_Coordinates_df], axis=1, join='inner')

In [24]:
toronto_neighborhoods.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M1G,43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,M1H,43.773136,-79.239476


In [25]:
toronto_neighborhoods.shape

(103, 6)